In [72]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

## Scraping Advanced Batting Data

In [73]:
def advancedBattingScraper(year):

  #fetching the HTML request object
  htmlData=requests.get(f'https://www.baseball-reference.com/leagues/majors/{year}-advanced-batting.shtml')

  #converting it into a Beautiful Soup object
  soupObject=BeautifulSoup(htmlData.text)

  #getting the HTML components using the nodes from the Selector Gadget Extension!
  hittingTableData=soupObject.select('#teams_advanced_batting tbody .left , #teams_advanced_batting tbody .right , #teams_advanced_batting .poptip')

  #converting the HTML components to just their values and storing them in an array
  advancedBattingDataAr=[]
  for element in hittingTableData:
    advancedBattingDataAr.append(element.text)

  #Dynamically getting all the headings of the dataframe
  TmIndex=advancedBattingDataAr.index('Tm')
  headings=advancedBattingDataAr[TmIndex:]

  #removing Teams since the Teams column shows up in column instead of row format (we deal with it later)
  headings.remove('Tm')
  
  #getting all the teams in one array
  indexToStop=advancedBattingDataAr.index('League Average')
  indices=advancedBattingDataAr[:indexToStop]

  #extracting just the data and reshaping a 1d array into a 2d array
  advancedBattingData=advancedBattingDataAr[indexToStop+1:-len(headings)*2-1]
  temp_arr=np.array(advancedBattingData).reshape(indexToStop,len(headings))

  #converting the array to a Pandas Dataframe and adding Year and Teams
  advancedBattingDf = pd.DataFrame(temp_arr, columns=headings)
  yearAr=np.full(len(indices),f'{year}')
  advancedBattingDf['Year']=yearAr
  advancedBattingDf['Tm']=indices

  #Dropping these two columns as they aren't present in all tables from 2001-2021
  if 'HardH%' in advancedBattingDf.columns:
    advancedBattingDf=advancedBattingDf.drop(['HardH%'], axis=1)
  if 'EV' in advancedBattingDf.columns:
    advancedBattingDf=advancedBattingDf.drop(['EV'], axis=1)
  return advancedBattingDf





In [74]:
TeamAdvancedBattingDf=advancedBattingScraper(2001)

for year in range(2002,2022):
  TeamAdvancedBattingDf=pd.concat([TeamAdvancedBattingDf,advancedBattingScraper(year)])

## Adding Playoff Data:

In [75]:
# Adding playoffs as a column

postseason_Request = requests.get('https://www.baseball-reference.com/postseason/')

#converting it into a Beautiful Soup object
postseason_soup_obj=BeautifulSoup(postseason_Request.text)

#getting the HTML components using the nodes from the Selector Gadget Extension!
postseason_data=postseason_soup_obj.select('#postseason_series a')
#postseason_data

In [76]:
seriesAr=[]
winnerAr=[]
loserAr=[]
for entry in range(0,len(postseason_data),3):
  seriesAr.append(postseason_data[entry].text)

for entry in range(1,len(postseason_data),3):
  winnerAr.append(postseason_data[entry].text)

for entry in range(2,len(postseason_data),3):
  loserAr.append(postseason_data[entry].text)

seriesAr=seriesAr[::-1]
winnerAr=winnerAr[::-1]
loserAr=loserAr[::-1]

def teamCleaner(x):
  x = x.split('(')[0]
  return x.split('*')[0].rstrip()

postseasonDf = pd.DataFrame([], columns=['Series', 'Winner', 'Loser'])
postseasonDf['Series']= list(map(lambda x: int(x.split(' ')[0]), seriesAr[4:]))
postseasonDf['Winner']=list(map(teamCleaner, winnerAr[4:]))
postseasonDf['Loser']=list(map(teamCleaner, loserAr[4:]))
postseasonDf

,Series,Winner,Loser
0,1884,Providence Grays,New York Metropolitans
1,1885,Chicago White Stockings,St. Louis Browns
2,1886,St. Louis Browns,Chicago White Stockings
3,1887,Detroit Wolverines,St. Louis Browns
4,1888,New York Giants,St. Louis Browns
...,...,...,...
390,2021,Boston Red Sox,Tampa Bay Rays
391,2021,Houston Astros,Chicago White Sox
392,2021,Atlanta Braves,Los Angeles Dodgers
393,2021,Houston Astros,Boston Red Sox


In [77]:
winnerYearDict= {}
yearAr=postseasonDf['Series'].unique()
for year in yearAr:
  arrayToAdd=[]
  arrayToAdd.append(postseasonDf.loc[postseasonDf['Series']==year]['Winner'].unique())
  arrayToAdd.append(postseasonDf.loc[postseasonDf['Series']==year]['Loser'].unique())
  winnerYearDict[year]=list(np.concatenate(arrayToAdd).flat)

winnerYearDict[2001]

['Arizona Diamondbacks',
 'Atlanta Braves',
 'New York Yankees',
 'Seattle Mariners',
 'St. Louis Cardinals',
 'Houston Astros',
 'Oakland Athletics',
 'Cleveland Indians',
 'Atlanta Braves',
 'Seattle Mariners',
 'New York Yankees']

In [78]:
TeamAdvancedBattingDf.reset_index(inplace=True)
playOffAr=['No']*len(TeamAdvancedBattingDf)
TeamAdvancedBattingDf['Playoff']=playOffAr

for index, row in TeamAdvancedBattingDf.iterrows():
  if row['Tm'] in winnerYearDict[int(row['Year'])]:
    TeamAdvancedBattingDf.loc[index, 'Playoff'] = 'Yes'

TeamAdvancedBattingDf.Playoff = TeamAdvancedBattingDf.Playoff.astype('category')
TeamAdvancedBattingDf

,index,rOBA,Rbat+,BAbip,ISO,HR%,SO%,BB%,LD%,GB%,...,Oppo%,WPA,cWPA,RE24,RS%,SB%,XBT%,Year,Tm,Playoff
0,0,.330,91,.290,.144,2.5%,16.1%,7.9%,25.5%,37.7%,...,20.1%,-10.0,-4.0%,-93.3,28%,69%,44%,2001,Anaheim Angels,No
1,1,.343,91,.294,.175,3.3%,16.6%,9.3%,24.6%,38.9%,...,19.7%,-2.9,-1.3%,28.4,31%,65%,42%,2001,Arizona Diamondbacks,Yes
2,2,.328,85,.290,.152,2.8%,16.9%,8.0%,22.6%,44.2%,...,21.2%,-5.9,-6.3%,-38.8,30%,65%,45%,2001,Atlanta Braves,Yes
3,3,.317,88,.278,.131,2.2%,16.1%,8.4%,24.6%,39.2%,...,19.9%,-10.0,-2.0%,-85.3,30%,72%,47%,2001,Baltimore Orioles,No
4,4,.341,100,.300,.173,3.2%,18.1%,8.3%,25.2%,40.2%,...,20.0%,-2.3,-1.5%,-19.2,30%,57%,34%,2001,Boston Red Sox,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,25,.322,101,.287,.168,3.3%,22.4%,8.0%,22.8%,40.5%,...,16.7%,1.2,-1.3%,3.6,30%,80%,43%,2021,St. Louis Cardinals,Yes
626,26,.328,112,.294,.186,3.6%,24.8%,9.4%,22.9%,42.4%,...,17.7%,8.0,6.1%,120.0,35%,68%,47%,2021,Tampa Bay Rays,Yes
627,27,.297,82,.280,.143,2.8%,23.2%,7.3%,22.5%,46.3%,...,18.5%,-16.7,-2.9%,-136.9,29%,79%,41%,2021,Texas Rangers,No
628,28,.345,113,.296,.200,4.3%,20.1%,8.2%,24.5%,40.5%,...,17.6%,3.7,2.9%,103.8,33%,80%,41%,2021,Toronto Blue Jays,No


## Cleaning Data

In [79]:
# converting the df to a csv file
TeamAdvancedBattingDf.to_csv('AdvancedBatting.csv', index= False)

In [80]:
dirty_data = pd.read_csv('AdvancedBatting.csv', float_precision='round_trip')
dirty_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630 entries, 0 to 629
Data columns (total 24 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   index    630 non-null    int64  
 1   rOBA     630 non-null    float64
 2   Rbat+    630 non-null    int64  
 3   BAbip    630 non-null    float64
 4   ISO      630 non-null    float64
 5   HR%      630 non-null    object 
 6   SO%      630 non-null    object 
 7   BB%      630 non-null    object 
 8   LD%      630 non-null    object 
 9   GB%      630 non-null    object 
 10  FB%      630 non-null    object 
 11  GB/FB    630 non-null    float64
 12  Pull%    630 non-null    object 
 13  Cent%    630 non-null    object 
 14  Oppo%    630 non-null    object 
 15  WPA      630 non-null    float64
 16  cWPA     630 non-null    object 
 17  RE24     630 non-null    float64
 18  RS%      630 non-null    object 
 19  SB%      630 non-null    object 
 20  XBT%     630 non-null    object 
 21  Year     630 non

In [81]:
dirty_data['HR%'].values[:10]

array(['2.5%', '3.3%', '2.8%', '2.2%', '3.2%', '3.1%', '3.5%', '2.8%',
       '3.3%', '3.3%'], dtype=object)

In [82]:
def cleanPercent(x):
  x=x[:-1]
  return float(x)/100

dirty_data['HR%'] = list(map(cleanPercent, dirty_data['HR%']))
dirty_data['SO%'] = list(map(cleanPercent, dirty_data['SO%']))
dirty_data['BB%'] = list(map(cleanPercent, dirty_data['BB%']))
dirty_data['LD%'] = list(map(cleanPercent, dirty_data['LD%']))
dirty_data['GB%'] = list(map(cleanPercent, dirty_data['GB%']))
dirty_data['FB%'] = list(map(cleanPercent, dirty_data['FB%']))
dirty_data['Pull%'] = list(map(cleanPercent, dirty_data['Pull%']))
dirty_data['Cent%'] = list(map(cleanPercent, dirty_data['Cent%']))
dirty_data['Oppo%'] = list(map(cleanPercent, dirty_data['Oppo%']))
dirty_data['cWPA'] = list(map(cleanPercent, dirty_data['cWPA']))
dirty_data['RS%'] = list(map(cleanPercent, dirty_data['RS%']))
dirty_data['SB%'] = list(map(cleanPercent, dirty_data['SB%']))
dirty_data['XBT%'] = list(map(cleanPercent, dirty_data['XBT%']))

In [83]:
dirty_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630 entries, 0 to 629
Data columns (total 24 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   index    630 non-null    int64  
 1   rOBA     630 non-null    float64
 2   Rbat+    630 non-null    int64  
 3   BAbip    630 non-null    float64
 4   ISO      630 non-null    float64
 5   HR%      630 non-null    float64
 6   SO%      630 non-null    float64
 7   BB%      630 non-null    float64
 8   LD%      630 non-null    float64
 9   GB%      630 non-null    float64
 10  FB%      630 non-null    float64
 11  GB/FB    630 non-null    float64
 12  Pull%    630 non-null    float64
 13  Cent%    630 non-null    float64
 14  Oppo%    630 non-null    float64
 15  WPA      630 non-null    float64
 16  cWPA     630 non-null    float64
 17  RE24     630 non-null    float64
 18  RS%      630 non-null    float64
 19  SB%      630 non-null    float64
 20  XBT%     630 non-null    float64
 21  Year     630 non

In [84]:
dirty_data = dirty_data.loc[dirty_data.Year!=2020]

## Converting clean data to csv file

In [85]:
# converting the clean df to a csv file
dirty_data.to_csv('AdvancedBattingClean.csv', index= False)